In [ ]:
import glob
files = glob.glob('data/Fire images/*.*')
ls_fire = []
for i in files:
	print([i.split('/')[1], '0'])
	ls_fire.append(['/', i.split('/')[1], '1'])

In [ ]:
import pandas as pd

df_fire = pd.DataFrame(ls_fire, columns=['Folder', 'filename', 'label'])

In [ ]:
df_fire.head()

In [ ]:
len(df_fire)

In [ ]:
df_fire['filename'].duplicated().sum()

In [ ]:
df_fire['filename'][0]

In [ ]:
df_fire['Folder'][0]

In [ ]:
from PIL import Image
from keras.preprocessing.image import img_to_array

img = Image.open(f'data/{df_fire["Folder"][0]}/{df_fire["filename"][0]}')
print(img.size)
image_red = img.resize((1024, 1024))
image = img_to_array(image_red)

In [ ]:
image

### Merge dfs

In [ ]:
import numpy as np

#Split df into train and test based on label column
df_train
df_test

def data_gen(df, batch_size):
	while True:
		x_batch = np.zeros((batch_size, 3, 1024, 1024))
		y_batch = np.zeros((batch_size, 1))
		for j in range(len(df/batch_size)):
			b = 0
			for m, k in zip(
				df['filename'].values[j*batch_size:(j+1)*batch_size],
				df['label'].values[j*batch_size:(j+1)*batch_size]
			):
				img = Image.open(f'{df["Folder"][b]}/{m}')
				image_red = img.resize((1024, 1024))
				x_batch[b] = img_to_array(image_red)
				y_batch[b] = k
				b += 1
			yield (x_batch, y_batch)


model.fit_generator(
	generator=data_gen(
		df_train,
		batch_size=batch_size
	), 
	steps_per_epoch=len(df_train) // batch_size, epochs=epochs
)